In [1]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim,\
                             mean_squared_error as mse,\
                             normalized_root_mse as nrmse,\
                             normalized_mutual_information as nmi


In [2]:
# Find the ERO-SNN folder and add it to the python path
current_dir = os.getcwd()

while os.path.basename(current_dir) != 'ERO-SNN':
    print(os.path.basename(current_dir))
    current_dir = os.path.dirname(current_dir)
    
print(f"Found ERO-SNN folder: {current_dir}")
sys.path.append(current_dir)
os.chdir(current_dir)

utils
Found ERO-SNN folder: /home/bernard/Desktop/IIT_Thesis/ERO-SNN


In [3]:
# Extracting the data
# datasets = ['h36m', 'EyeTracking', 'MVSEC']
datasets = ['h36m', 'MVSEC']

Number_of_Neighbours = [4, 8, 12, 20]
data_path = "Meshed Videos"
video_batches = {}
for dataset in datasets:
    video_batches[dataset] = {}
    for filename in os.listdir(data_path):
        if filename.startswith(dataset):
            print(f"Found {filename} in {dataset}")
            input_path = os.path.join(data_path, filename, "input-out.mp4")
            # eros_path = os.path.join(data_path, filename, "eros-out.mp4")
            for neighbours in Number_of_Neighbours:
                eros_path = os.path.join(data_path, filename, f"N{neighbours}", f"eros-out-N{neighbours}.mp4")
                scm_path = os.path.join(data_path, filename, f"N{neighbours}", "scm-out.mp4")
                video_batches[dataset][neighbours] = [input_path, eros_path, scm_path] 
                print(f"Found {len(video_batches[dataset][neighbours])} files for {dataset} with {neighbours} neighbours")

Found h36m_sample in h36m
Found 3 files for h36m with 4 neighbours
Found 3 files for h36m with 8 neighbours
Found 3 files for h36m with 12 neighbours
Found 3 files for h36m with 20 neighbours
Found MVSEC_short_outdoor in MVSEC
Found 3 files for MVSEC with 4 neighbours
Found 3 files for MVSEC with 8 neighbours
Found 3 files for MVSEC with 12 neighbours
Found 3 files for MVSEC with 20 neighbours


In [4]:
results = {dataset: {neighbours:{} for neighbours in Number_of_Neighbours} for dataset in datasets}

generate_scores = {'ssim_scores': True, 'mse_scores': False, 'nmse_scores': False, 'nmi_scores': True}

for dataset in datasets:
    for neighbours in Number_of_Neighbours:
        eros_path = video_batches[dataset][neighbours][1]
        scm_path = video_batches[dataset][neighbours][2]
        input_path = video_batches[dataset][neighbours][0]
        print(f"Dataset: {dataset}, Neighbours: {neighbours}")
        print(f"Processing combination:\n Input: {input_path},\n EROS: {eros_path},\n ESCM: {scm_path}\n")

        # Load videos
        input_video = cv2.VideoCapture(input_path)
        eros_video = cv2.VideoCapture(eros_path)
        scm_video = cv2.VideoCapture(scm_path)

        # Check if the videos are the same length and frame rate
        input_fps = input_video.get(cv2.CAP_PROP_FPS)
        eros_fps = eros_video.get(cv2.CAP_PROP_FPS)
        scm_fps = scm_video.get(cv2.CAP_PROP_FPS)
        
        input_frames = input_video.get(cv2.CAP_PROP_FRAME_COUNT)
        eros_frames = eros_video.get(cv2.CAP_PROP_FRAME_COUNT)
        scm_frames = scm_video.get(cv2.CAP_PROP_FRAME_COUNT)

        print(f"Input FPS: {input_fps}, EROS FPS: {eros_fps}, ESCM FPS: {scm_fps}")
        print(f"Input Frames: {input_frames}, EROS Frames: {eros_frames}, ESCM Frames: {scm_frames}")
        # continue
        ssim_scores = {'Input-EROS': [], 'EROS-ESCM': [], 'Input-ESCM': []}
        mse_scores = {'Input-EROS': [], 'EROS-ESCM': [], 'Input-ESCM': []}
        nmse_scores = {'Input-EROS': [], 'EROS-ESCM': [], 'Input-ESCM': []}
        nmi_scores = {'Input-EROS': [], 'EROS-ESCM': [], 'Input-ESCM': []}

        frame_index = 0
        
        while True:
            # Read frames from all videos
            ret1, frame1 = input_video.read()
            ret2, frame2 = eros_video.read()
            ret3, frame3 = scm_video.read()

            # Break the loop if any video ends
            if not ret1 or not ret2 or not ret3:
                print(f"End of video reached at frame {frame_index}.")
                break

            # Check if frames are in sync by comparing frame indices
            input_frame_idx = input_video.get(cv2.CAP_PROP_POS_FRAMES)
            eros_frame_idx = eros_video.get(cv2.CAP_PROP_POS_FRAMES)
            scm_frame_idx = scm_video.get(cv2.CAP_PROP_POS_FRAMES)

            if not (input_frame_idx == eros_frame_idx == scm_frame_idx == frame_index + 1):
                print(f"Error: Frames are out of sync at frame {frame_index}.")
                print(f"Input Frame Index: {input_frame_idx}, EROS Frame Index: {eros_frame_idx}, ESCM Frame Index: {scm_frame_idx}")
                break

            frame_index += 1

            # Convert frames to grayscale
            gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            gray3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)
            
            # # Apply binary threshold to convert to black and white
            # _, bw1 = cv2.threshold(gray1, 254, 255, cv2.THRESH_BINARY)
            # _, bw2 = cv2.threshold(gray2, 254, 255, cv2.THRESH_BINARY)
            # _, bw3 = cv2.threshold(gray3, 254, 255, cv2.THRESH_BINARY)
            
            bw1 = gray1
            bw2 = gray2
            bw3 = gray3

            # Compute SSIM between the two frames and append to the list
            if generate_scores['ssim_scores']:
                # ssim12, _ = ssim(bw1, bw2, full=True, data_range=255, use_sample_covariance=False, gaussian_weights=True, sigma=1.5)
                # ssim23, _ = ssim(bw2, bw3, full=True, data_range=255, use_sample_covariance=False, gaussian_weights=True, sigma=1.5)
                # ssim13, _ = ssim(bw1, bw3, full=True, data_range=255, use_sample_covariance=False, gaussian_weights=True, sigma=1.5)
                ssim12 = ssim(bw1, bw2)
                ssim23 = ssim(bw2, bw3)
                ssim13 = ssim(bw1, bw3)
                ssim_scores['Input-EROS'].append(ssim12)
                ssim_scores['EROS-ESCM'].append(ssim23)            
                ssim_scores['Input-ESCM'].append(ssim13)
            
            # Compute MSE between the frames and append to the list
            if generate_scores['mse_scores']:
                mse12 = mse(bw1, bw2)
                mse23 = mse(bw2, bw3)
                mse13 = mse(bw1, bw3)
                mse_scores['Input-EROS'].append(mse12)
                mse_scores['EROS-ESCM'].append(mse23)            
                mse_scores['Input-ESCM'].append(mse13)

            # Compute NMSE between the frames and append to the list
            if generate_scores['nmse_scores']:
                denom12 = np.sqrt(mse(bw1, bw1))
                denom23 = np.sqrt(mse(bw2, bw2))
                denom13 = np.sqrt(mse(bw3, bw3))
                
                if denom12 != 0:
                    nmse12 = nrmse(bw1, bw2)
                else:
                    nmse12 = float('inf')  # or some predefined value

                if denom23 != 0:
                    nmse23 = nrmse(bw2, bw3)
                else:
                    nmse23 = float('inf')  # or some predefined value

                if denom13 != 0:
                    nmse13 = nrmse(bw1, bw3)
                else:
                    nmse13 = float('inf')  # or some predefined value

                nmse_scores['Input-EROS'].append(nmse12)
                nmse_scores['EROS-ESCM'].append(nmse23)
                nmse_scores['Input-ESCM'].append(nmse13)
            
            # Compute NMI between the frames and append to the list
            if generate_scores['nmi_scores']:
                nmi12 = nmi(bw1, bw2)
                nmi23 = nmi(bw2, bw3)
                nmi13 = nmi(bw1, bw3)
                nmi_scores['Input-EROS'].append(nmi12)
                nmi_scores['EROS-ESCM'].append(nmi23)
                nmi_scores['Input-ESCM'].append(nmi13)
            
        # Release video captures
        input_video.release()
        eros_video.release()
        scm_video.release()
        
        # Create dataframes
        if generate_scores['ssim_scores']:
            results[dataset][neighbours]['ssim_scores'] = pd.DataFrame(ssim_scores)
        if generate_scores['mse_scores']:
            results[dataset][neighbours]['mse_scores'] = pd.DataFrame(mse_scores)
        if generate_scores['nmse_scores']:
            results[dataset][neighbours]['nmse_scores'] = pd.DataFrame(nmse_scores)
        if generate_scores['nmi_scores']:
            results[dataset][neighbours]['nmi_scores'] = pd.DataFrame(nmi_scores)

Dataset: h36m, Neighbours: 4
Processing combination:
 Input: Meshed Videos/h36m_sample/input-out.mp4,
 EROS: Meshed Videos/h36m_sample/N4/eros-out-N4.mp4,
 ESCM: Meshed Videos/h36m_sample/N4/scm-out.mp4

Input FPS: 33.0, EROS FPS: 33.0, ESCM FPS: 33.0
Input Frames: 1076.0, EROS Frames: 1065.0, ESCM Frames: 1076.0
End of video reached at frame 1065.
Dataset: h36m, Neighbours: 8
Processing combination:
 Input: Meshed Videos/h36m_sample/input-out.mp4,
 EROS: Meshed Videos/h36m_sample/N8/eros-out-N8.mp4,
 ESCM: Meshed Videos/h36m_sample/N8/scm-out.mp4

Input FPS: 33.0, EROS FPS: 33.0, ESCM FPS: 33.0
Input Frames: 1076.0, EROS Frames: 1065.0, ESCM Frames: 1076.0
End of video reached at frame 1065.
Dataset: h36m, Neighbours: 12
Processing combination:
 Input: Meshed Videos/h36m_sample/input-out.mp4,
 EROS: Meshed Videos/h36m_sample/N12/eros-out-N12.mp4,
 ESCM: Meshed Videos/h36m_sample/N12/scm-out.mp4

Input FPS: 33.0, EROS FPS: 33.0, ESCM FPS: 33.0
Input Frames: 1076.0, EROS Frames: 1065.0,

In [5]:
# Create a directory to save the results
output_dir = 'quantitative_results_matched_kernels'
os.makedirs(output_dir, exist_ok=True)

# Iterate through the results dictionary and save each DataFrame to a CSV file
for dataset_name, neighbors_dict in results.items():
    for n_neighbors, scores_dict in neighbors_dict.items():
        for score_type, df in scores_dict.items():
            # Define the filename
            filename = f"{dataset_name}_NN_{n_neighbors}_{score_type}.csv"
            filepath = os.path.join(output_dir, filename)
            
            # Save the DataFrame to a CSV file
            df.to_csv(filepath, index=False)
            print(f"Saved {filepath}")

Saved quantitative_results_matched_kernels/h36m_NN_4_ssim_scores.csv
Saved quantitative_results_matched_kernels/h36m_NN_4_nmi_scores.csv
Saved quantitative_results_matched_kernels/h36m_NN_8_ssim_scores.csv
Saved quantitative_results_matched_kernels/h36m_NN_8_nmi_scores.csv
Saved quantitative_results_matched_kernels/h36m_NN_12_ssim_scores.csv
Saved quantitative_results_matched_kernels/h36m_NN_12_nmi_scores.csv
Saved quantitative_results_matched_kernels/h36m_NN_20_ssim_scores.csv
Saved quantitative_results_matched_kernels/h36m_NN_20_nmi_scores.csv
Saved quantitative_results_matched_kernels/MVSEC_NN_4_ssim_scores.csv
Saved quantitative_results_matched_kernels/MVSEC_NN_4_nmi_scores.csv
Saved quantitative_results_matched_kernels/MVSEC_NN_8_ssim_scores.csv
Saved quantitative_results_matched_kernels/MVSEC_NN_8_nmi_scores.csv
Saved quantitative_results_matched_kernels/MVSEC_NN_12_ssim_scores.csv
Saved quantitative_results_matched_kernels/MVSEC_NN_12_nmi_scores.csv
Saved quantitative_results_ma

In [6]:
# Directory where the CSV files are saved
output_dir = 'quantitative_results_matched_kernels'

# List of score types
# score_types = ['ssim_scores', 'mse_scores', 'nmse_scores', 'nmi_scores']
# score_types = score_types[generate_scores.values()]

# DataFrame to store the statistics
stats_df = pd.DataFrame(columns=['Dataset', 'Neighbors', 'Score Type', 'Column', 'Mean', 'Std Dev', 'Min', 'Max'])

# Iterate through the CSV files in the directory
for filename in os.listdir(output_dir):
    if filename.endswith('scores.csv'):
        # Extract dataset name, number of neighbors, and score type from the filename
        parts = filename.split('_')
        dataset_name = parts[0]
        n_neighbors = parts[2]
        score_type = parts[3].split('.')[0]
        
        # Read the CSV file into a DataFrame
        filepath = os.path.join(output_dir, filename)
        df = pd.read_csv(filepath)
        
        # Compute statistics
        mean_values = df.mean()
        std_values = df.std()
        min_values = df.min()
        max_values = df.max()
        
        # Create a temporary DataFrame to hold the statistics for the current file
        temp_df = pd.DataFrame({
            'Dataset': dataset_name,
            'Neighbors': n_neighbors,
            'Score Type': score_type,
            'Column': df.columns,
            'Mean': mean_values.values,
            'Std Dev': std_values.values,
            'Min': min_values.values,
            'Max': max_values.values
        })
        
        # Concatenate the temporary DataFrame with the main stats_df
        stats_df = pd.concat([stats_df, temp_df], ignore_index=True)
        
        # Generate a plot for the DataFrame
        plt.figure(figsize=(10, 6))
        for column in df.columns:
            plt.plot(df[column], label=column)
        
        score_type = score_type.upper()
        plt.title(f'{score_type} scores for {dataset_name} with k = {n_neighbors}')
        plt.xlabel('Frame')
        plt.ylabel(score_type)
        plt.legend()
        plt.grid(True)
        
        # Save the plot
        plot_filename = f"{dataset_name}_NN_{n_neighbors}_{score_type}.png"
        plot_filepath = os.path.join(output_dir, plot_filename)
        plt.savefig(plot_filepath)
        plt.close()
        
        print(f"Saved plot {plot_filepath}")

# Save the statistics DataFrame to a CSV file
stats_filepath = os.path.join(output_dir, 'statistics_summary.csv')
stats_df.to_csv(stats_filepath, index=False)
print(f"Saved statistics summary to {stats_filepath}")

/tmp/ipykernel_924471/4106925733.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_df = pd.concat([stats_df, temp_df], ignore_index=True)


Saved plot quantitative_results_matched_kernels/MVSEC_NN_8_NMI.png
Saved plot quantitative_results_matched_kernels/MVSEC_NN_20_SSIM.png
Saved plot quantitative_results_matched_kernels/h36m_NN_4_NMI.png
Saved plot quantitative_results_matched_kernels/MVSEC_NN_12_NMI.png
Saved plot quantitative_results_matched_kernels/MVSEC_NN_12_SSIM.png
Saved plot quantitative_results_matched_kernels/h36m_NN_12_SSIM.png
Saved plot quantitative_results_matched_kernels/h36m_NN_20_NMI.png
Saved plot quantitative_results_matched_kernels/MVSEC_NN_20_NMI.png
Saved plot quantitative_results_matched_kernels/MVSEC_NN_4_NMI.png
Saved plot quantitative_results_matched_kernels/h36m_NN_8_NMI.png
Saved plot quantitative_results_matched_kernels/h36m_NN_20_SSIM.png
Saved plot quantitative_results_matched_kernels/h36m_NN_12_NMI.png
Saved plot quantitative_results_matched_kernels/h36m_NN_4_SSIM.png
Saved plot quantitative_results_matched_kernels/MVSEC_NN_8_SSIM.png
Saved plot quantitative_results_matched_kernels/MVSEC_N

In [8]:
output_dir = 'quantitative_results_matched_kernels'
stat_sum = pd.read_csv(os.path.join(output_dir, 'statistics_summary.csv'))

# Drop nmse_scores
stat_sum = stat_sum[stat_sum['Score Type'] != 'nmse']
# display(stat_sum)

grouped = stat_sum.groupby(['Dataset', 'Neighbors', 'Score Type', 'Column']).mean()
display(grouped)

Mean   Std Dev       Min  \
Dataset Neighbors Score Type Column                                     
MVSEC   4         nmi        EROS-ESCM   1.030249  0.041794  1.007172   
                             Input-EROS  1.003290  0.003652  1.000021   
                             Input-ESCM  1.013264  0.009644  1.000026   
                  ssim       EROS-ESCM   0.270750  0.218294  0.103552   
                             Input-EROS  0.045992  0.124680  0.002749   
                             Input-ESCM  0.194440  0.101477  0.080706   
        8         nmi        EROS-ESCM   1.024884  0.032563  1.006747   
                             Input-EROS  1.003856  0.003302  1.000021   
                             Input-ESCM  1.023025  0.013770  1.000025   
                  ssim       EROS-ESCM   0.260797  0.172619  0.123036   
                             Input-EROS  0.083535  0.116408  0.015513   
                             Input-ESCM  0.398220  0.076829  0.270025   
        12        nmi        EROS-ESCM   1.022327  0.028371  1.007247   
                             Input-EROS  1.004224  0.003338  1.000021   
                             Input-ESCM  1.030931  0.016641  1.000026   
                  ssim       EROS-ESCM   0.264700  0.148007  0.149209   
                             Input-EROS  0.131340  0.108591  0.029841   
                             Input-ESCM  0.530855  0.056909  0.425283   
        20        nmi        EROS-ESCM   1.019592  0.023523  1.006974   
                             Input-EROS  1.004914  0.003648  1.000021   
                             Input-ESCM  1.041982  0.020497  1.000025   
                  ssim       EROS-ESCM   0.323853  0.109470  0.243827   
                             Input-EROS  0.257242  0.098499  0.087656   
                             Input-ESCM  0.668807  0.035683  0.604763   
h36m    4         nmi        EROS-ESCM   1.075910  0.012431  1.061772   
                             Input-EROS  1.006895  0.009144  1.000000   
                             Input-ESCM  1.011237  0.023018  1.000000   
                  ssim       EROS-ESCM   0.802226  0.058819  0.714950   
                             Input-EROS  0.481517  0.155896  0.277450   
                             Input-ESCM  0.501446  0.152532  0.296637   
        8         nmi        EROS-ESCM   1.070768  0.011540  1.055578   
                             Input-EROS  1.007230  0.009852  1.000000   
                             Input-ESCM  1.014864  0.023265  1.000000   
                  ssim       EROS-ESCM   0.775227  0.071327  0.667714   
                             Input-EROS  0.490222  0.154082  0.286121   
                             Input-ESCM  0.557735  0.134694  0.377110   
        12        nmi        EROS-ESCM   1.068079  0.011415  1.051424   
                             Input-EROS  1.007390  0.009910  1.000000   
                             Input-ESCM  1.018151  0.024720  1.000000   
                  ssim       EROS-ESCM   0.753030  0.080519  0.631047   
                             Input-EROS  0.504751  0.149773  0.302902   
                             Input-ESCM  0.610698  0.116443  0.455192   
        20        nmi        EROS-ESCM   1.064689  0.011664  1.045264   
                             Input-EROS  1.007474  0.010702  1.000000   
                             Input-ESCM  1.023294  0.023696  1.000000   
                  ssim       EROS-ESCM   0.734489  0.088569  0.601675   
                             Input-EROS  0.543448  0.137501  0.353012   
                             Input-ESCM  0.685258  0.089285  0.569426   

                                              Max  
Dataset Neighbors Score Type Column                
MVSEC   4         nmi        EROS-ESCM   1.209533  
                             Input-EROS  1.089874  
                             Input-ESCM  1.196249  
                  ssim       EROS-ESCM   0.993931  
                             Input-EROS  0.995748  
                             I